# Data Preprocessing

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Load the dataset
df = pd.read_csv('../dataset/SriLanka_Supermarket_Sales.csv')

# Display the first few rows of the dataframe
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Product Code,Unit price,Quantity,Tax 5%,...,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,Inventory Level,Promotion,Restock Interval (Days)
0,278-37-5752,Matara,Matara,Member,Male,Meat,P009,LKR 302.33,4,LKR 60.47,...,6/12/2025,6:26:21 AM,NaN,"LKR 1,209.32",4.761905,LKR 60.47,8.0,118,Yes,7
1,483-53-5622,Galle,Galle,Normal,Female,Beverages,P002,LKR 234.51,10,LKR 117.25,...,10/28/2024,5:58:43 AM,Mobile Pay,"LKR 2,345.10",4.761905,LKR 117.25,8.9,104,No,10
2,562-74-1493,Colombo,Colombo,Normal,Male,Frozen Foods,P006,LKR 201.06,8,LKR 80.42,...,7/20/2025,4:30:06 PM,Credit Card,"LKR 1,608.48",4.761905,LKR 80.42,9.0,159,No,10
3,904-20-6783,Kandy,Kandy,Member,Male,Household,P003,LKR 279.93,6,LKR 83.98,...,8/21/2024,4:10:05 PM,E-wallet,"LKR 1,679.58",4.761905,LKR 83.98,8.8,101,No,10
4,382-87-2928,Colombo,Colombo,Normal,Male,Household,P003,LKR 414.51,2,LKR 41.45,...,5/18/2025,1:51:09 AM,E-wallet,LKR 829.02,4.761905,LKR 41.45,9.2,179,No,7


## 1. Data Cleaning

### Clean Currency Columns

In [3]:
currency_cols = ['Unit price', 'Tax 5%', 'Sales', 'cogs', 'gross income']
for col in currency_cols:
    df[col] = df[col].replace({'LKR ': '', ',': ''}, regex=True).astype(float)

### Handle Missing Values

In [4]:
# Impute 'Gender' with the mode
gender_imputer = SimpleImputer(strategy='most_frequent')
df['Gender'] = gender_imputer.fit_transform(df[['Gender']])

# Impute 'Payment' with the mode
payment_imputer = SimpleImputer(strategy='most_frequent')
df['Payment'] = payment_imputer.fit_transform(df[['Payment']])

# Impute 'Rating' with the mean
rating_imputer = SimpleImputer(strategy='mean')
df['Rating'] = rating_imputer.fit_transform(df[['Rating']])

# Check if there are any missing values left
df.isnull().sum()

Invoice ID                 0
Branch                     0
City                       0
Customer type              0
Gender                     0
Product line               0
Product Code               0
Unit price                 0
Quantity                   0
Tax 5%                     0
Sales                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
Inventory Level            0
Promotion                  0
Restock Interval (Days)    0
dtype: int64

## 2. Data Transformation

### Normalize Numerical Data

In [5]:
scaler = StandardScaler()
numerical_cols = df.select_dtypes(include=np.number).columns
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Product Code,Unit price,Quantity,Tax 5%,...,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,Inventory Level,Promotion,Restock Interval (Days)
0,278-37-5752,Matara,Matara,Member,Male,Meat,P009,0.028101,-0.534765,-0.400338,...,6/12/2025,6:26:21 AM,Credit Card,-0.400405,0.0,-0.400338,0.588846,0.138497,Yes,0.317847
1,483-53-5622,Galle,Galle,Normal,Female,Beverages,P002,-0.559466,1.556317,0.612959,...,10/28/2024,5:58:43 AM,Mobile Pay,0.613054,0.0,0.612959,1.127523,-0.128848,No,1.481340
2,562-74-1493,Colombo,Colombo,Normal,Male,Frozen Foods,P006,-0.849264,0.859290,-0.044310,...,7/20/2025,4:30:06 PM,Credit Card,-0.044234,0.0,-0.044310,1.187376,0.921435,No,1.481340
3,904-20-6783,Kandy,Kandy,Member,Male,Household,P003,-0.165964,0.162262,0.019222,...,8/21/2024,4:10:05 PM,E-wallet,0.019209,0.0,0.019222,1.067670,-0.186136,No,1.481340
4,382-87-2928,Colombo,Colombo,Normal,Male,Household,P003,0.999985,-1.231793,-0.739770,...,5/18/2025,1:51:09 AM,E-wallet,-0.739748,0.0,-0.739770,1.307082,1.303356,No,0.317847


### One-Hot Encode Categorical Data

In [6]:
categorical_cols = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df.head()

,Unit price,Quantity,Tax 5%,Sales,cogs,gross margin percentage,gross income,Rating,Inventory Level,Restock Interval (Days),...,Time_9:59:36 PM,Time_9:59:41 AM,Time_9:59:45 AM,Time_9:59:45 PM,Time_9:59:46 PM,Time_9:59:56 PM,Payment_Credit Card,Payment_E-wallet,Payment_Mobile Pay,Promotion_Yes
0,0.028101,-0.534765,-0.400338,-0.400402,-0.400405,0.0,-0.400338,0.588846,0.138497,0.317847,...,0,0,0,0,0,0,1,0,0,1
1,-0.559466,1.556317,0.612959,0.613050,0.613054,0.0,0.612959,1.127523,-0.128848,1.481340,...,0,0,0,0,0,0,0,0,1,0
2,-0.849264,0.859290,-0.044310,-0.044237,-0.044234,0.0,-0.044310,1.187376,0.921435,1.481340,...,0,0,0,0,0,0,1,0,0,0
3,-0.165964,0.162262,0.019222,0.019210,0.019209,0.0,0.019222,1.067670,-0.186136,1.481340,...,0,0,0,0,0,0,0,1,0,0
4,0.999985,-1.231793,-0.739770,-0.739749,-0.739748,0.0,-0.739770,1.307082,1.303356,0.317847,...,0,0,0,0,0,0,0,1,0,0
